In [1]:
import numpy as np

In [ ]:
# Equation 12
def distances(X, v, alpha, N, D, K):
    dists = np.zeros((N, K))
    for n in range(N):
        for i in range(D):
            for k in range(K):
                dists[n, k] += (X[n, i] - v[k, i]) * (X[n, i] - v[k, i]) * alpha[i]
    return dists

In [ ]:
# Equation 2&3
def M_nk(dists, N, K):
    M_nk = np.zeros((N, K))
    exp = np.zeros((N, K))
    denom = np.zeros(N)
    for n in range(N):
        for j in range(K):
            exp[n, j] = np.exp(-1 * dists[n, j])
            denom[n] += exp[n, j]
        for j in range(K):
            if denom[n]:
                M_nk[n, j] = exp[n, j] / denom[n]
            else:
                M_nk[n, j] = exp[n, j] / 1e-6
    return M_nk

In [ ]:
# Equation 6
def M_k(M_nk, N, K):
    M_k = np.zeros(K)
    for j in range(K):
        for n in range(N):
            M_k[j] += M_nk[n, j]
        M_k[j] /= N
    return M_k

In [ ]:
# Equation 8&9
def Loss_x(X, M_nk, v, N, D, K):
    x_n_hat = np.zeros((N, D))
    L_x = 0.0
    for n in range(N):
        for i in range(D):
            for k in range(K):
                x_n_hat[n, i] += M_nk[n, k] * v[k, i]
            L_x += (X[n, i] - x_n_hat[n, i]) * (X[n, i] - x_n_hat[n, i])
    return x_n_hat, L_x

In [ ]:
# Equation 10&11
def Loss_y(M_nk, y, w, N, K):
    yhat = np.zeros(N)
    L_y = 0.0
    for n in range(N):
        for k in range(K):
            yhat[n] += M_nk[n, k] * w[k]
        yhat[n] = 1e-6 if yhat[n] <= 0 else yhat[n]
        yhat[n] = 0.999 if yhat[n] >= 1 else yhat[n]
        L_y += -1 * y[n] * np.log(yhat[n]) - (1.0 - y[n]) * np.log(1.0 - yhat[n])
    return yhat, L_y

In [ ]:
# Equation 4 (target function)
def LFR(params, X_pos, X_neg, y_pos, y_neg, K, A_z, A_x, A_y, results=0):   
    #LFR.iters += 1
    N_pos, D = X_pos.shape # protected set
    N_neg, _ = X_neg.shape # non-protected set

    alpha_pos = params[:D]
    alpha_neg = params[D: 2 * D]
    w = params[2 * D: (2 * D) + K]
    v = np.matrix(params[(2 * D) + K:]).reshape((K, D)) # set of prototypes

    dists_pos = distances(X_pos, v, alpha_pos, N_pos, D, K)
    dists_neg = distances(X_neg, v, alpha_neg, N_neg, D, K)

    M_nk_pos = M_nk(dists_pos, N_pos, K)
    M_nk_neg = M_nk(dists_neg, N_neg, K)

    M_k_pos = M_k(M_nk_pos, N_pos, K)
    M_k_neg = M_k(M_nk_neg, N_neg, K)

    L_z = 0.0
    for k in range(K):
        L_z += abs(M_k_pos[k] - M_k_neg[k]) # Equation 7

    x_n_hat_pos, L_x_pos = Loss_x(X_pos, M_nk_pos, v, N_pos, D, K)
    x_n_hat_neg, L_x_neg = Loss_x(X_neg, M_nk_neg, v, N_neg, D, K)
    L_x = L_x_pos + L_x_neg

    yhat_pos, L_y_pos = Loss_y(M_nk_pos, y_pos, w, N_pos, K)
    yhat_neg, L_y_neg = Loss_y(M_nk_neg, y_neg, w, N_neg, K)
    L_y = L_y_pos + L_y_neg

    criterion = A_x * L_x + A_y * L_y + A_z * L_z

    #if LFR.iters % 250 == 0:
    #    print(LFR.iters, criterion)
    if results != 0:
        return yhat_pos, yhat_neg, M_nk_pos, M_nk_neg
    else:
        return criterion